<hr style="border:0.02in solid gray"> </hr>

<center><font color=#555555><font size=6> Laboratorio 4: Microestructura del mercado </font> <br> <br>
    
<center><font color=#555555><font size=5> Microestructura de trading </font> <br> <br> <br>

<center><font color=#555555><font size=5> Cuauhtémoc Corrales Camacho, if720482@iteso.mx </font> <br> <br>

<center><font color=#555555><font size=5> Jesús Sebastián González Ortiz, if721621@iteso.mx </font> <br> <br> <br>

<center> <font color= #555555> <font size = 4> Noviembre 2022 </a> | <font color= #555555> <font size = 4> Repository: <a href='https://github.com/SebasGlez2000/myst_lab4'>Link</a></font>

<hr style="border:0.02in solid gray"> </hr>

<font color= #555555> <font size = 4> Abstract </font>

Con la información recaudada de tres diferentes exchanges de diferentes partes del mundo, se usaron los libros de ordenes de tres diferentes criptomonedas que son BTC, ETH y SOL, con la información de volumenes y precios, el objetivo será visualizar y modelar la microestructura del mercado por medio de sus comportamientos de los libros de ordenes y sus características.

<br>

# <font color= #6B6B6B> <font size = 6> 1. Introduction </font>

<hr style="border:0.02in solid gray"> </hr>

En el siguiente laboratorio se estará trabajando en la Microestructura del mercado enfocado en las criptomonedas del momento, se recaurdará información del Libro de Ordenes de criptomonedas de diversos exchanges internacionales, esto con el propósito de visualizar y modelar la microestructura del mercado por medio de su comportamiento del libro de ordenes. 

Los libros de ordenes de dichas criptomonedas serán descargadas de tres exchanges de diferentes partes del mundo para ver la variación de precios que hay entre dichas plataformas y así poder observar si es posible el arbitraje entre ellos. Los exchanges utilizados serán Bitso que es una plataforma mexicana, Gate que es una plataforma de las Islas Caimán y Okcoin que es una plataforma de Estados Unidos. 

Como se puede observar, nuestros exchanges tienen sedes muy alejadas unas de otras entonces las ordenes recibidas entre dichos exchanges van a variar bastante unas de otras, la descarga del libro de ordenes se hará de una hora y de ahí se partirá para obtener los datos que se estan buscando para la realización del laboratorio.

De los Libros de Ordenes (LO) de los exchanges se extraerán los siguientes datos:

- Ask-volume para cada timestamp del LO: El ask-volume es la cantidad de ordenes que existen en el mercado para la **venta** del activo en el timestamp indicado.
- Bid-volume para cada timestamp del LO: El bid-volume es la cantidad de ordenes que existen en el mercado para la **compra** del activo en el timestamp indicado.
- Total Volume (ask-bid) para cada timestamp del LO: Es la suma de los volumenes que hay entre el bid y el ask en el timestamp indicado.
- Mid-Price (promedio de Top Of the Book) para cada timestamp del LO: Es el promedio de los precios de la oferta y la demanda del activo al momento.
- VWAP para cada timestamp del LO: Es el precio medio ponderado por volumen, muestra la relación entre el precio de un activo y su volumen total de operaciones.

A partir de los precios recolectados entre el bid y el ask, se calculará el spread, que es la diferencia de precios entre la demanda (bid) y la oferta (ask) de un mercado, y se contruirán series de tiempo con la información recopilada y procesada para su fácil visualización.

Se dividirá en tres secciones el laboratorio; el primero consiste en la extracción del Libro de Ordenes y OHLC de la criptomonedas de diversos exchanges, después la contrsucción de las Series de Tiempo indicadas previamente y por último es el modelado de la microestructura que consiste en estimar el Effective Spread del modelo de Roll.

<br>

# <font color= #6B6B6B> 2. Descripcion de datos </font>

<hr style="border:0.02in solid gray"> </hr>

 La información obtenida para el libro de ordenes son de tres criptomonedas que se consideran que son de las más influyentes del mercado: Ethereum (ETH), Solana (SOL) y Bitcoin (BTC), dichas monedas se medirán en dólares americanos para una visión más generalizada del mercado. Cada par sera extraido de los tres diferentes exchanges propuestos durante la introduccion, con lo cual se dispondran de nueve instrumentos para su analisis. 

 Esta informacion sera descargada de la paqueteria ccxt, la cual pide en su documentacion un instrumento, un exchange y un limite de ordenes. Y arroja como salidas un diccionario con el nombre del instrumento, los niveles de operaciones de bid y ask dentro del limite establecido, un timestamp en formato de microsegundos y una fecha en formata iso. 

 Para el desarrollo de este laboratorio se necesita extraer una hora de datos, donde dicho periodo de tiempo recopilara todos los instrumentos dentro del intervalo.

In [1]:
import visualizations as vs
import data as dt
import functions as fn

<br>

# <font color= #6B6B6B> 4. Procesamiento y Visualización de datos  </font>

<hr style="border:0.02in solid gray"> </hr>

El primer paso fue hacer una funcion de iteracion para almacenar los datos correspondientes a los nueve instrumentos en el periodo determinado de una hora. Dicha funcion tiene como entradas el limite de operaciones, el exchange y el tiempo asignado y arroja una lista con nueve grupos con los respectivos los datos en pares de segundos durante el intervalo. Dicha operacion de extraccion se realizo mediante la funcion cccxt.fetch_order_book.

Tras esto se creo la funcion de extracion de precios de cierre, la cual necesita de un exchange, un instrumento y un limite y arroja un diccionario que incluye los precios de cierre por minito en el limite escrito. La funcion utilizada para este metedo fue ccxt.fetch_ohlcv. Un dato curioso de este metodo es que extrae valores del datos mas nuevo hacia atras en el tiempo, en la longitud del limite ingresado. 

Con estas dos extracciones, se creo una funcion para crear un formato de salida como el especificado en la primera de las visualizaciones. 

## <font color= #6B6B6B> 4.1. Creacion de tabla de microestructura  </font>

La función llamada microestructura lee los datos descargados de las funciones anteriores para organizarlos en un DataFrame con el formato exigido en las indicaciones. Dicha funcion recibe como entrada una lista con los exchange, una lista con los instrumentos, el tiempo de intervalo y una lista de apoyo ya declarada al inicio del codigo. 

Dentro de esta funcion, se calcula el numero de niveles, el volumen total de operaciones *Bid* y *Ask*, asi como el volumen total operado; el precio medio del instrumento,  el cual se calcula mediante el promedio de los precios de *Bid* y *Ask* en su primer nivel; el VWAP se calcula mediante un promedio ponderado que implica los precios de *Bid* y *Ask* y sus respectivos volumenes. Finalmente se calcula el spread de precios para cada intervalo  y una columna de apoyo referente al tiempo del intervalo. 

In [2]:
tabla = fn.microestructura(dt.inst,50,3600,dt.exc,dt.exchl)
tabla.drop(['milliseconds','timestamp2','Spread'],axis=1)

,exchange,symbol,timestamp,level,ask_volume,bid_volume,total_volume,mid_price,VWAP
0,Bitso,ETH/USD,2022-11-12T03:59:57.000Z,50,97.937138,66.441732,164.378871,1257.300,1254.136119
1,Bitso,ETH/USD,2022-11-12T04:00:04.000Z,50,112.743717,66.644719,179.388437,1256.550,1254.130486
2,Bitso,ETH/USD,2022-11-12T04:00:10.000Z,50,118.619103,72.620153,191.239256,1256.000,1256.000000
3,Bitso,ETH/USD,2022-11-12T04:00:16.000Z,50,118.607873,70.259536,188.867409,1255.050,1254.408471
4,Bitso,ETH/USD,2022-11-12T04:00:22.000Z,50,112.509923,70.460333,182.970256,1255.200,1254.606922
...,...,...,...,...,...,...,...,...,...
5386,Okcoin,BTC/USD,2022-11-12T04:59:32.405Z,50,46.651100,68.525200,115.176300,16727.795,16732.806203
5387,Okcoin,BTC/USD,2022-11-12T04:59:38.629Z,50,53.579800,67.224700,120.804500,16726.740,16730.197143
5388,Okcoin,BTC/USD,2022-11-12T04:59:44.448Z,50,53.572200,67.358100,120.930300,16724.905,16718.997106
5389,Okcoin,BTC/USD,2022-11-12T04:59:50.249Z,50,39.058200,55.943000,95.001200,16712.440,16710.034828


## <font color= #6B6B6B> 4.2. Gráficas de instrumentos  </font>

Con la tabla de microestructura creada, es posible graficar el comportamiento de cada activo para el intervalo elegido. Para este caso, se graficara los cambios en el **Mid Price** o precio medio para cada uno de los nueve instrumentos. 

In [3]:
vs.grafos(tabla,dt.exchl)

El siguiente paso fue construir un formato para el calculo de spread efectivo, para el cual se definio una nueva funcion denominada como spread_cut, el cual recibia como entradas la tabla de microestructura y la tabla con los precios de cierre y arroja como salidas una lista con nueve tablas, las cuales tienen datos de timestamp en minutos, el precio de cierre y el spread real. 

La clave para este inciso fue establecer una union adecuada entre ambas tablas con el proposito de que cada activo y precio de cierre en la tabla de precios de cierre recibiera el spread real dentro de la tabla de micrestructura, ademas que se realizo una transformacion para que el timestamp de las salidas quedara en dentro del valor de minutos y no de segundos. 

Tras esto, se crearon dos funciones para el calculo del spread efectivo, la primera hace el calculo bruto del spread nuevo mediante la tabla obtenida en la funcion de microestructura, mientras que la segunda añade los resultados de la primera funcion y los concatena a las salidas de la funcion spread_cut mencionada anteriormente. 

In [4]:
# Close prices 
CPs = fn.close_prices(dt.inst,90,dt.exc)

# Tablas Spreads
ppp = fn.spread_cut(tabla,CPs,dt.exchl,dt.inst)

# Calculo de Effective Spread
es = fn.EffectiveSpread(tabla)

# Resultados finales
FTable = fn.ESR(ppp, es)
Resultados = fn.Salidas(FTable)

<br>

# <font color= #6B6B6B> <font size = 6> 5. Resultados </font>

<hr style="border:0.02in solid gray"> </hr>

Como salidas del nuestros procesos, se obtienen nueve tablas, las cuales almacenan la informacion del timestamp en su indice, el precio de cierre en su primera columna, el spread real en su segunda columna y el resultado del calculado del spread efectivo en su tercera columna. 

## <font color= #6B6B6B> 5.1 Resultados ETH/USD </font>

In [5]:
Resultados[0].head()

,CP ETH/USD Bitso,Spread,Effective,CP ETH/USD Gate.io,Spread,Effective,CP ETH/USD OKCoin,Spread,Effective
2022-11-12T03:59,1255.6,6.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-12T04:00,1255.7,4.9,0.385232,1257.16,0.27,0.232236,1256.13,1.01,0.722713
2022-11-12T04:01,1257.3,2.3,0.492446,1259.84,0.25,2.030727,1258.01,1.63,0.453458
2022-11-12T04:02,1259.6,6.5,0.523686,1260.53,2.41,1.248356,1259.84,1.57,0.282487
2022-11-12T04:03,1259.6,3.1,3.130361,1261.18,0.24,0.125474,1259.84,1.78,0.434028


## <font color= #6B6B6B> 5.2 Resultados SOL/USD </font>

In [6]:
Resultados[1].head()

,CP SOL/USD Bitso,Spread,Effective,CP SOL/USD Gate.io,Spread,Effective,CP SOL/USD OKCoin,Spread,Effective
2022-11-12T03:59,15.777,0.029,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-12T04:00,15.744,0.027,0.010855,15.761,0.031,0.009052,15.777,0.106,0.043265
2022-11-12T04:01,15.781,0.029,0.012858,15.761,0.041,0.009833,15.777,0.138,0.006545
2022-11-12T04:02,15.784,0.019,0.001751,15.761,0.041,0.000817,15.777,0.129,0.026639
2022-11-12T04:03,15.784,0.024,0.002855,15.851,0.044,0.015668,15.777,0.133,0.030763


## <font color= #6B6B6B> 5.3 Resultados BTC/USD </font>

In [7]:
Resultados[2].head()

,CP BTC/USD Bitso,Spread,Effective,CP BTC/USD Gate.io,Spread,Effective,CP BTC/USD OKCoin,Spread,Effective
timestamp2,,,,,,,,,
2022-11-12T04:00,16863.0,9.0,1.368201,16881.38,20.91,9.388415,16880.61,11.15,3.859320
2022-11-12T04:01,16863.0,7.0,3.678377,16889.37,0.53,6.686121,16882.19,10.68,2.298522
2022-11-12T04:02,16879.0,10.0,4.088825,16890.93,11.73,5.810308,16882.19,13.50,0.977632
2022-11-12T04:03,16891.0,6.0,3.976031,16890.40,1.05,1.990791,16882.19,7.70,2.076966
2022-11-12T04:04,16880.0,35.0,2.044129,16876.66,0.01,5.958753,16877.12,9.37,0.553869


## <font color= #6B6B6B> 5.4 Interpretación </font>

A primera instancia, se observa que la relacion entre el spread real y el spread efectivo es diferente para cada par. Mientras Ethereum y Solana tienen un spread real mayor al efectivo, hay un gran numero de casos para Bitcoin donde el caso es el opuesto, el spread real es menor al efectivo. 

La diferencia entre el spread real y el spread efectivo es que el spread real se enfoca en la diferencia que existe entre el valor *ask* mas barato y el valor *bid* mas caro, y sirve como referencia a la posible ganancia al trader, Por otro lado, el spread efectivo se enfoca en comparar precios de cierre con precios medios, con el proposito de ver si el spread efectivo otorga un mayor rendimiento dinamico a los traders. Es decir, el spread efectivo te provee del costo que un tarder ha de pagar para tener una mejora en dicha operacion. 

Con esto en cuenta, se concluye que si el spread efectivo es mayor al spread real, no conviene realizar una operacion par, ya que bajo este concepto, el spread real es un referente hacia la ganancia del trader, y de ser mayor, la ganancia generada por el spread real queda nulificada. En caso contrario, si el spread real es mayor al spread efectivo, entonces la diferencia entre ambos en la rentabilidad minima de dicha operacion. 

Aplicando esto a las criptomonedas elegidas, Ethereum y Solana presentan un costo viable hacia la posibilidad de mejorar operaciones, mientras que el costo de mejora de Bitcoin no es viable en los casos donde su spread efectivo supera al real. Con esto, se motiva a los traders a invertir en ETH o SOL si quisieran tener mejores prospectos de inversion. 

<br>

# <font color= #6B6B6B> <font size = 6> 6. References </font>

<hr style="border:0.02in solid gray"> </hr>

- VWAP (definición). (s. f.). IG. https://www.ig.com/es/glosario-trading/definicion-de-vwap

- elEconomista.es. (s. f.). Spread: qué es - Diccionario de Economía. https://www.eleconomista.es/diccionario-de-economia/spread

- Nine, E. N. (2022, 7 enero). El precio BID, el MID y el ASK. ¿Qué son y para qué se usan? ». Ninety Nine. https://ninetynine.com/blog/bid-mid-ask/

<br>